## Curso Python para Economistas
### Trabajo Práctico Nº 6 (integrador)

### Fecha de entrega:
Lunes 13/11 a las 23:59 hs\*

\* *tienen una semana más que lo usual para resolver este TP*

### Modalidad de entrega y trabajo
- Este TP es **individual**.
- Un repositorio **privado** debe ser creado en GitHub para el TP, y dar acceso a los 5 profesores. Los nombres de usuarios son: `belenmichel`, `rbonazzola`, `Queeno11`, `agoszulli` y `pilarch`.
- Cuando hayan hecho ese último commit, copien la URL para clonar su repositorio y péguenla en
[este Google Sheet](https://docs.google.com/spreadsheets/d/160DBUCMAMUPtpeWtRl0SJYyYd2Miv-sYuFQZO3J3MS0/edit?usp=sharing), en la hoja del TP6. Al ser un repositorio privado, solo los colaboradores habilitados podrán clonarlo.
- Al finalizar el trabajo práctico deben hacer un último commit y push en su repositorio de GitHub con el mensaje `"Entrega final del TP6"`. Antes de la fecha y hora de entrega pueden hacer cuantos cambios quieran en el repositorio, pero luego de la hora de corte no deben hacer más cambios. Si un commit con el mensaje anterior se realiza luego de la hora de entrega, se supondrá que la entrega tardía fue intencional y se utilizarán los días de gracia. La última versión en el repositorio es la que será evaluada. Para esto es importante que no completen el Google Sheet hasta que no hayan finalizado el TP, como tampoco hacer pushes posteriores a la entrega.
- El estilo del código tendrá un peso en la nota de este trabajo (pequeño pero no nulo). Pueden utilizar esta [guía de estilo](https://recursospython.com/pep8es.pdf) como referencia.

## Objetivo

El objetivo de este trabajo es visualizar en un mapa los resultados electorales de las elecciones generales que tuvieron lugar el pasado 22 de octubre. 

Va a incluir los temas de la semana 6 (datos georreferenciados y mapas), pero al ser integrador también necesitarán revisar conceptos de semanas previas, en particular cómo realizar consultas a una base de datos SQLite, cómo operar con dataframe's de Pandas, cómo realizar gráficos usando distintas librerías, y trabajar con widgets interactivos.

## Datos

En este trabajo vamos a utilizar dos conjuntos de datos, llamémoslos `elecciones` y `jurisdicciones`.


`elecciones` se lo damos en un archivo en formato SQLite y contiene los resultados de las elecciones a nivel de mesa.

`jurisdicciones` son dos archivos georreferenciados que **no** están incluidos en este repositorio: los tienen que bajar ustedes mismos (ver sección correspondiente abajo).

#### `elecciones`

Esta base de datos está en el archivo `resultados_electorales.sqlite` que pueden bajarse de [este link de Google Drive](https://drive.google.com/file/d/1wiNTv7uUuuSFZLOdaIPAB7UFkmLByTKC/view?usp=sharing) y está compuesta por distintas tablas:
- La tabla principal con las cuentas de los votos por mesa para cada partido y cargo, llamada `resultados`: tiene como columnas de `mesa_id`, `seccion_id`, `circuito_id` , `agrupacion_id`, `cargo_id` (presidente/vice, diputados o senadores) y finalmente la cuenta de los votos, `cantidad`. Cada partido político puede tener varias listas, pero el partido político **no** aparece explícitamente en esta tabla (se puede, sin embargo, deducir a partir de la lista). El sufijo `_id` se debe a que los valores no son el nombre de la entidad que representan, sino códigos numéricos. Se hace así por razones de eficiencia en el almacenamiento y en la consulta: es más eficiente almacenar un entero que un string como `"FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD"`.

- Los códigos están vinculados al nombre en sí a través de otra tabla de la base de datos. Entonces, tenemos las tablas `distritos`, `secciones`, `partidos` y `cargos`.

Es recomendable examinar la base de datos `resultados_electorales.sqlite`: mirar, en particular, los nombres de las tablas, y los nombres de las columnas en cada tabla. Esto lo pueden realizar utilizando un programa como [SQLiteBrowser](https://sqlitebrowser.org/dl/).

#### `jurisdicciones`
Estos son dos archivos con datos georreferenciados que contienen las coordenadas de las distintas jurisdicciones adonde queremos reportar los resultados. Estas jurisdicciones son 

- los **departamentos**, llamados "secciones" en las tablas anteriores. Este shapefile fue utilizado ya en la clase sincrónica 6, y había sido bajado del [sitio del IGN](https://www.ign.gob.ar/NuestrasActividades/InformacionGeoespacial/CapasSIG), <br>
- los **circuitos electorales**, que deben bajar de [esta página](https://mapa2.electoral.gov.ar/descargas/). Hay distintos formatos disponibles, pueden usar el que prefieran. Pueden elegir cualquier provincia (en el inciso 2 les pedimos que filtren los datos para una provincia).

## Consignas

1. Utilizando el paquete `sqlite3` de Python, realizar una consulta (_query_) del archivo `resultados_electorales.sqlite`, cuyo resultado sea la unión de la tabla de `resultados` con las tablas `agrupaciones` por las columnas correspondientes; para de esa manera tener en una misma tabla la correspondencia con el partido político. Guardar el resultado de esta _query_ en una variable. Examinar la estructura del resultado. ¿Qué tipo de dato es? Convertir esta estructura en un dataframe de Pandas (si no lo es ya).
<br> _Aclaración:_ notar que el resultado de la consulta de SQL ya debe darnos como resultado la unión de las tablas de resultados y agrupaciones. A pesar de que podrían se unir las tablas en Pandas (usando `pd.merge`) y daría el mismo resultado, no daremos la puntuación máxima si lo hacen de este modo. (Si bien esto puede parecer un capricho, hay muchos casos en que es conveniente delegar en un servidor remoto de base de datos el procesamiento de datos voluminosos y sólo realizar el procesamiento posterior localmente (en Python), que típicamente involucraría operaciones más sofisticadas pero que no requieren tanta memoria o capacidad de procesamiento.)

In [389]:
import sqlite3
import pandas as pd

# Base de datos SQLite
conexion = sqlite3.connect('resultados_electorales.sqlite')

# Consulta SQL para unir las tablas
consulta_sql = '''
    SELECT r.*, a.*
    FROM resultados r
    INNER JOIN agrupaciones a ON r.agrupacion_id = a.agrupacion_id
'''

# Guardo el resultado en un DataFrame
resultado_query = pd.read_sql_query(consulta_sql, conexion)

# Primeras filas del DataFrame resultante
resultado_query.head()


,distrito_id,seccion_id,circuito_id,mesa_id,cargo_id,agrupacion_id,votos_cantidad,agrupacion_id,agrupacion_nombre
0,1,1,00018,475,1,134,95,134,UNION POR LA PATRIA
1,1,1,00018,475,1,132,59,132,JUNTOS POR EL CAMBIO
2,1,1,00018,475,1,135,57,135,LA LIBERTAD AVANZA
3,1,1,00018,475,1,136,9,136,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD
4,1,1,00018,475,1,133,4,133,HACEMOS POR NUESTRO PAIS


2. En Pandas, unir el dataframe que obtuvieron con las tablas de `distritos`, `secciones` y `cargos` del archivo. Luego, eliminar las columnas que representan los identificadores originales para estas entidades, así como para las agrupaciones políticas (las que terminan en `"_id"`). Finalmente, elegir una provincia y conservar los resultados sólo para esa provincia. Debe ser la misma para la que se bajaron el shapefile de circuitos electorales.

In [390]:
# Tablas de distritos, secciones y cargos desde el SQLite
distritos = pd.read_sql_query('SELECT * FROM distritos', conexion)
secciones = pd.read_sql_query('SELECT * FROM secciones', conexion)
cargos = pd.read_sql_query('SELECT * FROM cargos', conexion)

# Uno el df resultado_query con las tablas de distritos, secciones y cargos
resultado_merged = pd.merge(resultado_query, distritos, how= 'inner')
resultado_merged = pd.merge(resultado_merged, secciones, how= 'inner')
resultado_merged = pd.merge(resultado_merged, cargos, how= 'inner')
resultado_merged
# Elimino columnas
columnas_eliminar = ['distrito_id', 'seccion_id','cargo_id', 'agrupacion_id']
resultado_final = resultado_merged.drop(columns=columnas_eliminar)
resultado_merged
resultado_final


,circuito_id,mesa_id,votos_cantidad,agrupacion_nombre,distrito_nombre,seccion_nombre,cargo_nombre
0,00018,475,95,UNION POR LA PATRIA,Ciudad Autónoma de Buenos Aires,Comuna 01,PRESIDENTE Y VICE
1,00018,475,59,JUNTOS POR EL CAMBIO,Ciudad Autónoma de Buenos Aires,Comuna 01,PRESIDENTE Y VICE
2,00018,475,57,LA LIBERTAD AVANZA,Ciudad Autónoma de Buenos Aires,Comuna 01,PRESIDENTE Y VICE
3,00018,475,9,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,Ciudad Autónoma de Buenos Aires,Comuna 01,PRESIDENTE Y VICE
4,00018,475,4,HACEMOS POR NUESTRO PAIS,Ciudad Autónoma de Buenos Aires,Comuna 01,PRESIDENTE Y VICE
...,...,...,...,...,...,...,...
1151151,01800,554,4,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,Santa Cruz,Güer Aike,SENADOR NACIONAL
1151152,01800,555,48,UNION POR LA PATRIA,Santa Cruz,Güer Aike,SENADOR NACIONAL
1151153,01800,555,41,POR SANTA CRUZ,Santa Cruz,Güer Aike,SENADOR NACIONAL
1151154,01800,555,35,CAMBIA SANTA CRUZ,Santa Cruz,Güer Aike,SENADOR NACIONAL


In [391]:
# Resultados solo para la provincia elegida
provincia_elegida = 'Salta'
resultado_provincia = resultado_final[resultado_final['distrito_nombre'] == provincia_elegida]

# Primeras filas del df
resultado_provincia.head()

,circuito_id,mesa_id,votos_cantidad,agrupacion_nombre,distrito_nombre,seccion_nombre,cargo_nombre
358885,0053C,3777,16,HACEMOS POR NUESTRO PAIS,Salta,Cerrillos,PRESIDENTE Y VICE
358886,0053C,3777,5,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,Salta,Cerrillos,PRESIDENTE Y VICE
358887,0053C,3778,132,LA LIBERTAD AVANZA,Salta,Cerrillos,PRESIDENTE Y VICE
358888,0053C,3778,106,UNION POR LA PATRIA,Salta,Cerrillos,PRESIDENTE Y VICE
358889,0053C,3778,26,HACEMOS POR NUESTRO PAIS,Salta,Cerrillos,PRESIDENTE Y VICE


In [392]:
from unidecode import unidecode
resultado_provincia['seccion_nombre'] = resultado_provincia['seccion_nombre'].apply(unidecode)
resultado_provincia.head()

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\3330887061.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,circuito_id,mesa_id,votos_cantidad,agrupacion_nombre,distrito_nombre,seccion_nombre,cargo_nombre
358885,0053C,3777,16,HACEMOS POR NUESTRO PAIS,Salta,Cerrillos,PRESIDENTE Y VICE
358886,0053C,3777,5,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,Salta,Cerrillos,PRESIDENTE Y VICE
358887,0053C,3778,132,LA LIBERTAD AVANZA,Salta,Cerrillos,PRESIDENTE Y VICE
358888,0053C,3778,106,UNION POR LA PATRIA,Salta,Cerrillos,PRESIDENTE Y VICE
358889,0053C,3778,26,HACEMOS POR NUESTRO PAIS,Salta,Cerrillos,PRESIDENTE Y VICE


3. En el dataframe de arriba tenemos los resultados para presidente/vice, diputados y senadores en filas distintas (son distintas _observaciones_ según la nomenclatura usual). Reformatear el dataframe de manera que contenga tres columnas nuevas, llamadas `presidente`, `diputados` y `senadores`, donde en cada una deben estar los resultados de cada categoría para cada mesa.

In [393]:

#columna 'categoria' para diferenciar entre presidente, diputados y senadores
resultado_provincia['categoria'] = resultado_provincia['cargo_nombre'].str.lower()

# Agrupo por las columnas relevantes y la nueva columna 'categoria'
resultado_provincia_grouped = resultado_provincia.groupby(['circuito_id', 'mesa_id', 'distrito_nombre', 'seccion_nombre','agrupacion_nombre','categoria'])['votos_cantidad'].sum().reset_index()
resultado_provincia_grouped
# Utilizo pivot_table para reorganizar el df
resultado_provincia_reformateado = resultado_provincia_grouped.pivot_table(
    values='votos_cantidad', 
    index=['circuito_id', 'mesa_id', 'agrupacion_nombre','distrito_nombre', 'seccion_nombre'], 
    columns='categoria', 
    aggfunc='sum'
    ).reset_index()

# valores NaN 
resultado_provincia_reformateado = resultado_provincia_reformateado.fillna(0)

# Mostrar el df reformateado
resultado_provincia_reformateado

# Renombrar las columnas
resultado_provincia_reformateado = resultado_provincia_reformateado.rename(columns={
    'diputado nacional': 'diputados',
    'presidente y vice': 'presidente'
})
resultado_provincia_reformateado.head(10)

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\3487826483.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



categoria,circuito_id,mesa_id,agrupacion_nombre,distrito_nombre,seccion_nombre,diputados,presidente
0,0001A,1001,AHORA PATRIA,Salta,Capital,86.0,0.0
1,0001A,1001,AUTONOMISTA,Salta,Capital,13.0,0.0
2,0001A,1001,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,Salta,Capital,16.0,8.0
3,0001A,1001,HACEMOS POR NUESTRO PAIS,Salta,Capital,0.0,15.0
4,0001A,1001,JUNTOS POR EL CAMBIO,Salta,Capital,79.0,89.0
5,0001A,1001,LA LIBERTAD AVANZA,Salta,Capital,0.0,99.0
6,0001A,1001,SI - SALTA INDEPENDIENTE,Salta,Capital,11.0,0.0
7,0001A,1001,UNION POR LA PATRIA,Salta,Capital,36.0,41.0
8,0001A,1002,AHORA PATRIA,Salta,Capital,80.0,0.0
9,0001A,1002,AUTONOMISTA,Salta,Capital,14.0,0.0


4. Construir dos dataframes, `partidos_circ_df` y `partidos_dpto_df`: en uno tendremos la cantidad de votos para cada partido político por circuito electoral; y el otro será equivalente, pero para cada departamento. (Si eligieron la provincia de Buenos Aires, serían partidos en lugar de departamentos, pero pueden usar la misma nomenclatura.)

In [394]:
partidos_circ_df =resultado_provincia_reformateado.groupby(['circuito_id', 'agrupacion_nombre']).agg({'diputados': 'sum', 'presidente':'sum'}).reset_index() 
partidos_circ_df

categoria,circuito_id,agrupacion_nombre,diputados,presidente
0,0001A,AHORA PATRIA,1208.0,0.0
1,0001A,AUTONOMISTA,216.0,0.0
2,0001A,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,116.0,70.0
3,0001A,HACEMOS POR NUESTRO PAIS,0.0,261.0
4,0001A,JUNTOS POR EL CAMBIO,1102.0,1235.0
...,...,...,...,...
2603,0103A,HACEMOS POR NUESTRO PAIS,0.0,5.0
2604,0103A,JUNTOS POR EL CAMBIO,8.0,7.0
2605,0103A,LA LIBERTAD AVANZA,0.0,118.0
2606,0103A,SI - SALTA INDEPENDIENTE,3.0,0.0


In [395]:
partidos_dpto_df = resultado_provincia_reformateado.groupby(['seccion_nombre', 'agrupacion_nombre']).agg({'diputados': 'sum', 'presidente':'sum'}).reset_index()
partidos_dpto_df.head(20)


categoria,seccion_nombre,agrupacion_nombre,diputados,presidente
0,Anta,AHORA PATRIA,11308.0,0.0
1,Anta,AUTONOMISTA,1050.0,0.0
2,Anta,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,251.0,253.0
3,Anta,HACEMOS POR NUESTRO PAIS,0.0,1148.0
4,Anta,JUNTOS POR EL CAMBIO,3676.0,3945.0
5,Anta,LA LIBERTAD AVANZA,0.0,11863.0
6,Anta,SI - SALTA INDEPENDIENTE,546.0,0.0
7,Anta,UNION POR LA PATRIA,15919.0,16596.0
8,Cachi,AHORA PATRIA,2137.0,0.0
9,Cachi,AUTONOMISTA,197.0,0.0


5. Usando Geopandas, cargar los shapefiles de circuitos electorales y departamentos en sendos geodataframes. Unir (`merge`) el primero con `partidos_circ_df`; y el segundo con `partidos_dpto_df`. Los resultados deberían ser dos geodataframes (pueden reemplazar el sufijo `df` por `gdf` en el nombre).
Para hacer la unión, deben decidir qué columnas utilizar como criterio de unión. Deben examinar los valores de las mismas y prestar particular atención a si es necesario realizar alguna transformación previa en alguna tabla.

In [396]:
import geopandas as gpd
provincias_gdf = gpd.read_file(filename="datos/provincia.zip") 
provincias_gdf = provincias_gdf[["nam" , "geometry"]]

provincias_gdf.explore()

#salta
salta_gdf = provincias_gdf[provincias_gdf['nam'] == 'Salta']
salta_gdf.explore()

In [397]:
dptos_gdf = gpd.read_file(filename= "datos/departamento.zip", crs='EPSG:4326')
dptos_gdf = dptos_gdf[["objectid" , "nam","geometry"]]
dptos_gdf = dptos_gdf.rename({"nam":"departamento"}, axis=1)
dptos_gdf['departamento'] = dptos_gdf['departamento'].apply(unidecode)
dptos_gdf.head()

,objectid,departamento,geometry
0,8076.0,Federal,"POLYGON ((-58.57731 -30.69016, -58.57512 -30.6..."
1,8077.0,Gualeguaychu,"POLYGON ((-58.13011 -33.02912, -58.13087 -33.0..."
2,8078.0,Islas del Ibicuy,"POLYGON ((-58.57589 -34.02675, -58.57593 -34.0..."
3,8268.0,Cushamen,"POLYGON ((-71.17490 -41.99995, -71.05132 -41.9..."
4,8079.0,Victoria,"POLYGON ((-59.77158 -32.58791, -59.77198 -32.5..."


In [398]:
dptos_centroid_gdf = dptos_gdf.copy()
dptos_centroid_gdf["geometry"] = dptos_gdf.geometry.centroid

sla_dep_gdf= dptos_centroid_gdf.sjoin(salta_gdf, predicate="within").reset_index()
sla_dep_gdf= sla_dep_gdf[["objectid", "geometry", "departamento"]]
sla_dep_gdf.head()
sla_dep_gdf.departamento.unique()

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\3091566802.py:2: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




array(['La Candelaria', 'Rosario de la Frontera', 'Guachipas', 'Iruya',
       'Cachi', 'Chicoana', 'La Vina', 'Metan', 'Molinos', 'Cerrillos',
       'General Guemes', 'San Carlos', 'La Caldera', 'Anta', 'Los Andes',
       'Oran', 'Cafayate', 'Capital', 'La Poma', 'Santa Victoria',
       'General Jose de San Martin', 'Rivadavia', 'Rosario de Lerma'],
      dtype=object)

In [399]:
dptos_unido = dptos_gdf.sjoin(sla_dep_gdf, predicate="intersects")
dptos_unido = dptos_unido.rename({"departamento_left": "departamento", "objectid_left":"objectid"}, axis=1)
dptos_unido = dptos_unido[["objectid", "geometry", "departamento"]]
dptos_unido.head()


,objectid,geometry,departamento
15,8407.0,"POLYGON ((-64.94161 -26.30225, -64.94161 -26.3...",La Candelaria
16,8304.0,"POLYGON ((-64.19751 -25.70929, -64.20483 -25.7...",Rosario de la Frontera
42,8409.0,"POLYGON ((-65.18657 -25.40954, -65.18657 -25.4...",Guachipas
68,8208.0,"POLYGON ((-64.58476 -22.67584, -64.58475 -22.6...",Iruya
73,8116.0,"POLYGON ((-65.85396 -25.07102, -65.85396 -25.0...",Cachi


In [400]:
unido_prov= dptos_unido.sjoin(salta_gdf, predicate="intersects")
unido_prov.head()

,objectid,geometry,departamento,index_right,nam
15,8407.0,"POLYGON ((-64.94161 -26.30225, -64.94161 -26.3...",La Candelaria,20,Salta
16,8304.0,"POLYGON ((-64.19751 -25.70929, -64.20483 -25.7...",Rosario de la Frontera,20,Salta
42,8409.0,"POLYGON ((-65.18657 -25.40954, -65.18657 -25.4...",Guachipas,20,Salta
68,8208.0,"POLYGON ((-64.58476 -22.67584, -64.58475 -22.6...",Iruya,20,Salta
73,8116.0,"POLYGON ((-65.85396 -25.07102, -65.85396 -25.0...",Cachi,20,Salta


In [401]:
departamentos = unido_prov.merge(partidos_dpto_df, left_on= "departamento", right_on="seccion_nombre", how="inner")
departamentos
departamentos = departamentos[["objectid", "geometry", "departamento", "seccion_nombre", "agrupacion_nombre" ,"diputados", "presidente"]]
departamentos.head(10)
departamentos.departamento.unique()

array(['La Candelaria', 'Rosario de la Frontera', 'Guachipas', 'Iruya',
       'Cachi', 'Chicoana', 'La Vina', 'Metan', 'Molinos', 'Cerrillos',
       'General Guemes', 'San Carlos', 'La Caldera', 'Anta', 'Los Andes',
       'Oran', 'Cafayate', 'Capital', 'Santa Victoria',
       'General Jose de San Martin', 'Rivadavia', 'Rosario de Lerma'],
      dtype=object)

In [402]:
votos_deptos = departamentos.groupby(by=['seccion_nombre', 'geometry', "agrupacion_nombre"]).agg({'presidente':'sum', 'diputados' : 'sum'}).reset_index()
votos_deptos = votos_deptos.pivot(index=['seccion_nombre'  , 'geometry'], columns='agrupacion_nombre', values =["presidente", "diputados"]).fillna(0).reset_index()

votos_deptos.columns= [' '.join(col).strip() for col in votos_deptos.columns.values]
votos_deptos=votos_deptos.rename(columns={'': 'geometry'})

#gdf
votos_deptos= gpd.GeoDataFrame(votos_deptos, geometry= 'geometry')
votos_deptos = votos_deptos.rename(columns={'seccion_nombre': 'departamento'})
votos_deptos

,departamento,geometry,presidente AHORA PATRIA,presidente AUTONOMISTA,presidente FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,presidente HACEMOS POR NUESTRO PAIS,presidente JUNTOS POR EL CAMBIO,presidente LA LIBERTAD AVANZA,presidente SI - SALTA INDEPENDIENTE,presidente UNION POR LA PATRIA,diputados AHORA PATRIA,diputados AUTONOMISTA,diputados FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,diputados HACEMOS POR NUESTRO PAIS,diputados JUNTOS POR EL CAMBIO,diputados LA LIBERTAD AVANZA,diputados SI - SALTA INDEPENDIENTE,diputados UNION POR LA PATRIA
0,Anta,"POLYGON ((-63.53042 -24.12787, -63.53043 -24.1...",0.0,0.0,253.0,1148.0,3945.0,11863.0,0.0,16596.0,11308.0,1050.0,251.0,0.0,3676.0,0.0,546.0,15919.0
1,Cachi,"POLYGON ((-65.85396 -25.07102, -65.85396 -25.0...",0.0,0.0,94.0,226.0,761.0,2244.0,0.0,2058.0,2137.0,197.0,108.0,0.0,728.0,0.0,101.0,1968.0
2,Cafayate,"POLYGON ((-65.72622 -25.87282, -65.72621 -25.8...",0.0,0.0,189.0,929.0,1199.0,3748.0,0.0,4018.0,3518.0,762.0,236.0,0.0,1020.0,0.0,349.0,3658.0
3,Capital,"POLYGON ((-65.22012 -24.71597, -65.20448 -24.7...",0.0,0.0,8091.0,26430.0,57663.0,146818.0,0.0,96286.0,137422.0,20884.0,10644.0,0.0,52044.0,0.0,12849.0,84627.0
4,Cerrillos,"POLYGON ((-65.25805 -25.16157, -65.25807 -25.1...",0.0,0.0,593.0,2091.0,4304.0,15011.0,0.0,10195.0,14108.0,1670.0,869.0,0.0,3918.0,0.0,991.0,8907.0
5,Chicoana,"POLYGON ((-65.53171 -25.02690, -65.53166 -25.0...",0.0,0.0,193.0,782.0,1968.0,6755.0,0.0,4574.0,6539.0,671.0,206.0,0.0,1752.0,0.0,481.0,4207.0
6,General Guemes,"POLYGON ((-64.64009 -24.61118, -64.64151 -24.6...",0.0,0.0,514.0,1671.0,3580.0,12719.0,0.0,13032.0,11903.0,1413.0,515.0,0.0,3296.0,0.0,733.0,11973.0
7,General Jose de San Martin,"POLYGON ((-63.13219 -21.99933, -63.13219 -22.0...",0.0,0.0,907.0,3095.0,8236.0,29339.0,0.0,50297.0,27882.0,2646.0,1005.0,0.0,7760.0,0.0,1268.0,47313.0
8,Guachipas,"POLYGON ((-65.18657 -25.40954, -65.18657 -25.4...",0.0,0.0,21.0,89.0,322.0,1038.0,0.0,924.0,1016.0,76.0,22.0,0.0,296.0,0.0,54.0,900.0
9,Iruya,"POLYGON ((-64.58476 -22.67584, -64.58475 -22.6...",0.0,0.0,2.0,28.0,40.0,463.0,0.0,717.0,454.0,25.0,2.0,0.0,40.0,0.0,9.0,701.0


In [403]:
circuitos_gdf = gpd.read_file(filename="datos/circuito_17.shp")
circuitos_gdf = circuitos_gdf[["departamen","circuito", "geometry"]]
circuitos_gdf

,departamen,circuito,geometry
0,Capital,0002A,"POLYGON ((-65.42374 -24.78894, -65.42544 -24.7..."
1,Capital,0002Q,"POLYGON ((-65.37932 -24.79907, -65.37938 -24.7..."
2,Cerrillos,0054A,"POLYGON ((-65.49459 -24.97446, -65.49458 -24.9..."
3,Capital,0003H,"POLYGON ((-65.43752 -24.78803, -65.43752 -24.7..."
4,La Viña,0060,"POLYGON ((-65.76462 -25.48014, -65.75744 -25.4..."
...,...,...,...
171,Capital,0003A,"POLYGON ((-65.48066 -24.74967, -65.48034 -24.7..."
172,La Caldera,0009,"POLYGON ((-65.22118 -24.71619, -65.22294 -24.7..."
173,Cerrillos,0052,"POLYGON ((-65.47443 -24.87079, -65.47433 -24.8..."
174,Capital,0002Q,"POLYGON ((-65.38041 -24.82443, -65.38045 -24.8..."


In [404]:
circuitos_gdf.departamen.unique()

array(['Capital', 'Cerrillos', 'La Viña', 'La Poma',
       'General Jose de San Martin', 'Anta', 'Oran', 'Guachipas',
       'Santa Victoria', 'Rivadavia', 'Rosario de Lerma', 'Los Andes',
       'Metan', 'Rosario de la Frontera', 'La Candelaria', 'Cachi',
       'San Carlos', 'Iruya', 'Chicoana', 'General Güemes', 'Molinos',
       'Cafayate', 'La Caldera'], dtype=object)

In [405]:
partidos_circ_df['circuito_id'] = partidos_circ_df['circuito_id'].astype(str)
circuitos_gdf['circuito']= circuitos_gdf['circuito'].astype(str)

circuitos_gdf['circuito']= circuitos_gdf['circuito'].str.zfill(5)
circuitos = circuitos_gdf.merge(partidos_circ_df, left_on='circuito', right_on='circuito_id', how='inner')
circuitos.head()

,departamen,circuito,geometry,circuito_id,agrupacion_nombre,diputados,presidente
0,Capital,0002A,"POLYGON ((-65.42374 -24.78894, -65.42544 -24.7...",0002A,AHORA PATRIA,2186.0,0.0
1,Capital,0002A,"POLYGON ((-65.42374 -24.78894, -65.42544 -24.7...",0002A,AUTONOMISTA,322.0,0.0
2,Capital,0002A,"POLYGON ((-65.42374 -24.78894, -65.42544 -24.7...",0002A,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,160.0,100.0
3,Capital,0002A,"POLYGON ((-65.42374 -24.78894, -65.42544 -24.7...",0002A,HACEMOS POR NUESTRO PAIS,0.0,530.0
4,Capital,0002A,"POLYGON ((-65.42374 -24.78894, -65.42544 -24.7...",0002A,JUNTOS POR EL CAMBIO,1102.0,1190.0


In [406]:
votos_circ = circuitos.groupby(by= ["circuito", "departamen", "geometry", "agrupacion_nombre"]).agg({'presidente':'sum', 'diputados' : 'sum'}).reset_index()

votos_circ = votos_circ.pivot(index=['circuito', 'departamen', 'geometry'], columns= 'agrupacion_nombre', values=['presidente', 'diputados']).fillna(0).reset_index()
votos_circ.columns = ['  '.join(col).strip() for col in votos_circ.columns.values]
votos_circ= votos_circ.rename(columns={'': 'geometry'})
votos_circ= votos_circ.rename(columns={'departamen': 'departamento'})

#gdf
votos_circ= gpd.GeoDataFrame(votos_circ, geometry= 'geometry')
votos_circ.head()


,circuito,departamento,geometry,presidente AHORA PATRIA,presidente AUTONOMISTA,presidente FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,presidente HACEMOS POR NUESTRO PAIS,presidente JUNTOS POR EL CAMBIO,presidente LA LIBERTAD AVANZA,presidente SI - SALTA INDEPENDIENTE,presidente UNION POR LA PATRIA,diputados AHORA PATRIA,diputados AUTONOMISTA,diputados FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,diputados HACEMOS POR NUESTRO PAIS,diputados JUNTOS POR EL CAMBIO,diputados LA LIBERTAD AVANZA,diputados SI - SALTA INDEPENDIENTE,diputados UNION POR LA PATRIA
0,0001A,Capital,"POLYGON ((-65.43105 -24.78844, -65.43105 -24.7...",0.0,0.0,70.0,261.0,1235.0,1264.0,0.0,776.0,1208.0,216.0,116.0,0.0,1102.0,0.0,165.0,678.0
1,0001B,Capital,"POLYGON ((-65.41626 -24.78908, -65.41564 -24.7...",0.0,0.0,15.0,73.0,438.0,261.0,0.0,133.0,255.0,66.0,19.0,0.0,373.0,0.0,36.0,104.0
2,0001C,Capital,"POLYGON ((-65.39287 -24.79541, -65.39384 -24.7...",0.0,0.0,42.0,154.0,1007.0,895.0,0.0,426.0,875.0,155.0,72.0,0.0,918.0,0.0,112.0,365.0
3,0001D,Capital,"POLYGON ((-65.41518 -24.77837, -65.41496 -24.7...",0.0,0.0,34.0,150.0,1019.0,890.0,0.0,453.0,850.0,136.0,62.0,0.0,910.0,0.0,104.0,397.0
4,0001E,Capital,"POLYGON ((-65.40598 -24.77056, -65.40154 -24.7...",0.0,0.0,70.0,219.0,794.0,1071.0,0.0,701.0,1004.0,185.0,104.0,0.0,729.0,0.0,119.0,561.0


6. Crear un mapa interactivo donde se puedan visualizar los resultados electorales. Se utilizarán dos widgets, que inducirán el siguiente comportamiento:
- Debe haber un widget que permita elegir la jurisdicción a mostrar: esta jurisdicción puede ser, o bien toda la provincia, o bien un departamento de la provincia (es decir que, si $N$ es el número de departamentos, el widget debe mostrar $N+1$ opciones). Si el usuario elige toda la provincia, se debe mostrar un mapa de la misma dividido en departamentos. Si se elige un departamento, se debe mostrar un mapa del departamento elegido, dividido en circuitos electorales. 
- Otro widget determina si se muestran los resultados para presidente/vice, para senadores o para diputados.

In [407]:
import folium
import ipywidgets as widgets
from ipywidgets import interactive

def display_map(jurisdiccion, categoria):
    if categoria == 'Senadores':
        print('No hay datos para Senadores 2023')
    else:
        # Crear un mapa 
        mapa = folium.Map(location=[-25, -66], zoom_start=6, name='Elecciones Generales 2023')

        partido_prefix = ""
        circuitos_data = None

        # Si se selecciona 'Toda la provincia'
        if jurisdiccion == 'Toda la provincia':
            data = votos_deptos

            # Agregar votos según la categoría seleccionada
            if categoria == 'Presidente':
                data['votos'] = data.filter(like='presidente').sum(axis=1)
                partido_prefix = 'presidente '
            elif categoria == 'Senadores':
                data['votos'] = data.filter(like='senadores').sum(axis=1)
                partido_prefix = 'senadores '
            elif categoria == 'Diputados':
                data['votos'] = data.filter(like='diputados').sum(axis=1)
                partido_prefix = 'diputados '

            # Crear un mapa de coropletas
            folium.Choropleth(
                geo_data=data,
                data=data,
                columns=['departamento', 'votos'],
                key_on='feature.properties.departamento',
                fill_color='YlOrBr',  
                fill_opacity=0.9,
                line_opacity=1,
                legend_name='Cantidad de votos por departamento',
                highlight=True,  
            ).add_to(mapa)

            # Agregar información detallada al pasar el cursor
            style_function = lambda x: {'fillColor': 'white', 'color': 'black'} 
            info = folium.features.GeoJson(
                data,
                style_function=style_function,
                control=False,
                tooltip=folium.features.GeoJsonTooltip(
                    fields=['departamento', 'votos'] + data.columns[data.columns.str.contains(partido_prefix)].tolist(),
                    aliases=['Departamento', 'Votos totales'] + [f'{partido} Votos' for partido in data.columns[data.columns.str.contains(partido_prefix)].tolist()],
                    labels=True,
                    sticky=True
                )
            )
            mapa.add_child(info)

            # Agregar icono al hacer clic en la región
            for _, row in data.iterrows():
                folium.Marker(
                    location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
                    popup=f"<b>{row['departamento']}<br>Votos totales: {int(row['votos'])}</b>",
                    icon=folium.Icon(color='red', icon='info-sign')  
                ).add_to(mapa)

        else:
            # Si se selecciona un departamento específico
            data = votos_deptos[votos_deptos['departamento'] == jurisdiccion]

            # Verificar si hay datos de circuitos para el departamento seleccionado
            if jurisdiccion in votos_circ['departamento'].unique():
                circuitos_data = votos_circ[votos_circ['departamento'] == jurisdiccion]

                # Agregar votos según la categoría seleccionada para los circuitos
                if categoria == 'Presidente':
                    circuitos_data['votos'] = circuitos_data.filter(like='presidente').sum(axis=1)
                    partido_prefix = 'presidente '
                elif categoria == 'Senadores':
                    circuitos_data['votos'] = circuitos_data.filter(like='senadores').sum(axis=1)
                    partido_prefix = 'senadores '
                elif categoria == 'Diputados':
                    circuitos_data['votos'] = circuitos_data.filter(like='diputados').sum(axis=1)
                    partido_prefix = 'diputados '

            # Crear un mapa para los circuitos
            if circuitos_data is not None:
                folium.Choropleth(
                    geo_data=circuitos_data,
                    data=circuitos_data,
                    columns=['circuito', 'votos'],
                    key_on='feature.properties.circuito',
                    fill_color='YlOrBr',  
                    fill_opacity=0.9,
                    line_opacity=1,
                    legend_name='Cantidad de votos por circuito',
                    highlight=True,  
                ).add_to(mapa)

                # información detallada al pasar el cursor
                style_function = lambda x: {'fillColor': 'white', 'color': 'blue'}  # Estilo de resaltado
                info = folium.features.GeoJson(
                    circuitos_data,
                    style_function=style_function,
                    control=False,
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['circuito', 'votos'] + circuitos_data.columns[circuitos_data.columns.str.contains(partido_prefix)].tolist(),
                        aliases=['Circuito', 'Votos totales'] + [f'{partido} Votos' for partido in circuitos_data.columns[circuitos_data.columns.str.contains(partido_prefix)].tolist()],
                        labels=True,
                        sticky=True
                    )
                )
                mapa.add_child(info)

                # Agregar icono al hacer clic en la región
                for _, row in circuitos_data.iterrows():
                    folium.Marker(
                        location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
                        popup=f"<b>{row['circuito']}<br>Votos totales: {int(row['votos'])}</b>",
                        icon=folium.Icon(color='red', icon='info-sign')
                    ).add_to(mapa)

        # Mostrar el mapa
        display(mapa)

# Widgets
jurisdiccion_widget = widgets.Dropdown(
    options=['Toda la provincia'] + list(votos_deptos['departamento'].unique()),
    description='Jurisdicción:')

categoria_widget = widgets.Dropdown(
    options=['Presidente', 'Senadores', 'Diputados'],
    description='Categoría:')

# Crear un widget interactivo
interactive_map = interactive(display_map, jurisdiccion=jurisdiccion_widget, categoria=categoria_widget)

# widget interactivo
display(interactive_map)



interactive(children=(Dropdown(description='Jurisdicción:', options=('Toda la provincia', 'Anta', 'Cachi', 'Ca…

7. Ahora vamos a examinar los resultados para presidente/vice para las tres fuerzas mayoritarias: Unión por la Patria (UPP), La Libertad Avanza (LLA) y Juntos por el Cambio (JPC). Existe una forma de representar ternas de valores que suman 1: los [gráficos ternarios](https://es.wikipedia.org/wiki/Diagrama_ternario). Son gráficos con forma de triángulo equilátero, donde los vértices corresponden a que uno de los valores de la terna es igual a 1. <br><br>

En este ejercicio, vamos a generar plots ternarios que representen la fracción de votos para cada una de las tres agrupaciones anteriores. Crear dos widgets de selección: uno que permita elegir un departamento de la provincia, y el otro que determine si se muestran los resultados a nivel de mesa o a nivel de circuito electoral. Usando estos parámetros, mostrar de manera reactiva un gráfico ternario. _Se evaluará la presentación del gráfico_. <br><br>

Para generar el gráfico ternario pueden utilizar la librería que deseen. Una opción es la librería `plotly`, que no vimos. Si usan `plotly`, [esta](https://plotly.com/python/ternary-plots/) es la documentación que pueden usar de referencia. Otra opción es una extensión de `matplotlib` llamada [mpltern](https://mpltern.readthedocs.io/en/latest/installation.html). En ambos casos deberían instalar las librerías.

El plot ternario debe ser tal que:
- Contenga puntos (scatter plot) donde los valores sean las fracciones de cada una de los tres partidos.
- Un punto represente los resultados para una mesa electoral o para un circuito electoral, dependiendo del valor del segundo widget. 
- El tamaño del punto esté dado por la cantidad total de electores que votaron a los tres partidos en esa mesa o ese circuito.
- El color del punto esté determinado por qué partido sacó mayoría: vamos a usar azul para UPP, amarillo para JPC y violeta para LLA.

_Nota_: los gráficos ternarios suponen que los valores suman 1 (o una constante), por lo tanto los valores se deben normalizar a la cantidad de votos que sacaron las tres fuerzas consideradas (no al total de votos de la mesa). 

In [408]:
nivel_depto = votos_deptos[['departamento'] + votos_deptos.filter(like='presidente').columns.tolist()]
nivel_depto.columns = nivel_depto.columns.str.replace('presidente', '', regex=True)
nivel_depto.rename ({' JUNTOS POR EL CAMBIO': 'JPC', ' LA LIBERTAD AVANZA':'LLA', ' UNION POR LA PATRIA':'UPP'} , axis=1, inplace=True)
nivel_depto.drop(columns={' FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD'}, axis=1, inplace=True)
nivel_depto.drop(columns={' HACEMOS POR NUESTRO PAIS'}, axis=1, inplace=True)
nivel_depto.drop(columns={' AHORA PATRIA'}, axis=1, inplace=True)
nivel_depto.drop(columns={' AUTONOMISTA'}, axis=1, inplace=True)
nivel_depto.drop(columns={' SI - SALTA INDEPENDIENTE'}, axis=1, inplace=True)



nivel_depto

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\3664434109.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\3664434109.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\3664434109.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\3664434109.py:6: SettingWithCopyWarning:


A value

,departamento,JPC,LLA,UPP
0,Anta,3945.0,11863.0,16596.0
1,Cachi,761.0,2244.0,2058.0
2,Cafayate,1199.0,3748.0,4018.0
3,Capital,57663.0,146818.0,96286.0
4,Cerrillos,4304.0,15011.0,10195.0
5,Chicoana,1968.0,6755.0,4574.0
6,General Guemes,3580.0,12719.0,13032.0
7,General Jose de San Martin,8236.0,29339.0,50297.0
8,Guachipas,322.0,1038.0,924.0
9,Iruya,40.0,463.0,717.0


In [409]:
nivel_depto['total']= nivel_depto[['JPC', 'LLA','UPP']].sum(axis=1)
prop_columns = ['Juntos por el Cambio', 'La Libertad Avanza', 'Union por la patria']
vote_columns = ['JPC', 'LLA', 'UPP']
nivel_depto[prop_columns]= nivel_depto[vote_columns].div(nivel_depto['total'], axis=0)
nivel_depto


C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\3463494366.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\3463494366.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\3463494366.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,departamento,JPC,LLA,UPP,total,Juntos por el Cambio,La Libertad Avanza,Union por la patria
0,Anta,3945.0,11863.0,16596.0,32404.0,0.121744,0.366097,0.512159
1,Cachi,761.0,2244.0,2058.0,5063.0,0.150306,0.443215,0.406478
2,Cafayate,1199.0,3748.0,4018.0,8965.0,0.133742,0.418070,0.448187
3,Capital,57663.0,146818.0,96286.0,300767.0,0.191720,0.488145,0.320135
4,Cerrillos,4304.0,15011.0,10195.0,29510.0,0.145849,0.508675,0.345476
5,Chicoana,1968.0,6755.0,4574.0,13297.0,0.148003,0.508009,0.343987
6,General Guemes,3580.0,12719.0,13032.0,29331.0,0.122055,0.433637,0.444308
7,General Jose de San Martin,8236.0,29339.0,50297.0,87872.0,0.093727,0.333883,0.572389
8,Guachipas,322.0,1038.0,924.0,2284.0,0.140981,0.454466,0.404553
9,Iruya,40.0,463.0,717.0,1220.0,0.032787,0.379508,0.587705


In [410]:
niv_circuito = votos_circ[['departamento'] +['circuito']+ votos_circ.filter(like='presidente').columns.tolist()]
niv_circuito.columns = niv_circuito.columns.str.replace('presidente', '', regex=True)
niv_circuito.rename ({'  JUNTOS POR EL CAMBIO': 'JPC', '  LA LIBERTAD AVANZA':'LLA', '  UNION POR LA PATRIA':'UPP'} , axis=1, inplace=True)
niv_circuito.drop(columns={'  FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD'}, axis=1, inplace=True)
niv_circuito.drop(columns={'  HACEMOS POR NUESTRO PAIS'}, axis=1, inplace=True)
niv_circuito.drop(columns={'  AHORA PATRIA'}, axis=1, inplace=True)
niv_circuito.drop(columns={'  AUTONOMISTA'}, axis=1, inplace=True)
niv_circuito.drop(columns={'  SI - SALTA INDEPENDIENTE'}, axis=1, inplace=True)
niv_circuito

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\1611567833.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\1611567833.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\1611567833.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\1611567833.py:6: SettingWithCopyWarning:


A value

,departamento,circuito,JPC,LLA,UPP
0,Capital,0001A,1235.0,1264.0,776.0
1,Capital,0001B,438.0,261.0,133.0
2,Capital,0001C,1007.0,895.0,426.0
3,Capital,0001D,1019.0,890.0,453.0
4,Capital,0001E,794.0,1071.0,701.0
...,...,...,...,...,...
80,Cerrillos,0054A,783.0,3260.0,2434.0
81,Los Andes,0100A,273.0,1612.0,1461.0
82,Los Andes,0101A,11.0,62.0,66.0
83,Los Andes,0102A,0.0,0.0,0.0


In [411]:
niv_circuito['total']= niv_circuito[['JPC', 'LLA','UPP']].sum(axis=1)
prop_columns = ['Juntos por el Cambio', 'La Libertad Avanza', 'Union por la patria']
vote_columns = ['JPC', 'LLA', 'UPP']
niv_circuito[prop_columns]= niv_circuito[vote_columns].div(niv_circuito['total'], axis=0)
niv_circuito

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\3635852020.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\3635852020.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\3635852020.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,departamento,circuito,JPC,LLA,UPP,total,Juntos por el Cambio,La Libertad Avanza,Union por la patria
0,Capital,0001A,1235.0,1264.0,776.0,3275.0,0.377099,0.385954,0.236947
1,Capital,0001B,438.0,261.0,133.0,832.0,0.526442,0.313702,0.159856
2,Capital,0001C,1007.0,895.0,426.0,2328.0,0.432560,0.384450,0.182990
3,Capital,0001D,1019.0,890.0,453.0,2362.0,0.431414,0.376799,0.191787
4,Capital,0001E,794.0,1071.0,701.0,2566.0,0.309431,0.417381,0.273188
...,...,...,...,...,...,...,...,...,...
80,Cerrillos,0054A,783.0,3260.0,2434.0,6477.0,0.120889,0.503319,0.375791
81,Los Andes,0100A,273.0,1612.0,1461.0,3346.0,0.081590,0.481769,0.436641
82,Los Andes,0101A,11.0,62.0,66.0,139.0,0.079137,0.446043,0.474820
83,Los Andes,0102A,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [412]:
import plotly.express as px
from ipywidgets import widgets, interactive

def determine_ganador(row):
    if row['La Libertad Avanza'] > row['Juntos por el Cambio'] and row['La Libertad Avanza'] > row['Union por la patria']:
        return 'LLA'
    elif row['Juntos por el Cambio'] > row['La Libertad Avanza'] and row['Juntos por el Cambio'] > row['Union por la patria']:
        return 'JPC'
    else:
        return 'UPP'

nivel_depto['ganador'] = nivel_depto.apply(determine_ganador, axis=1)
niv_circuito['ganador'] = niv_circuito.apply(determine_ganador, axis=1)

def generate_ternary_plot(departamento, nivel):
    if nivel == "Departamento":
        df = nivel_depto
    elif nivel == "Circuito":
        df = niv_circuito   
    df = df[df['departamento'] == departamento]
    fig = px.scatter_ternary(df, a='Juntos por el Cambio', b='La Libertad Avanza', c='Union por la patria', size='total', hover_data=['total'], title=f'RESULTADOS GENERALES: {departamento}')
    color_map = {'LLA': 'purple', 'JPC': 'yellow', 'UPP': 'blue'}
    fig.update_traces(marker=dict(color=[color_map[ganador] for ganador in df['ganador']]))
    fig.show()

departamento_widget = widgets.Dropdown(
    options=nivel_depto['departamento'].unique(),
    description='Departamento')

nivel_widget = widgets.Dropdown(
    options=['Departamento', 'Circuito'],
    description='Nivel:')

interactive_plot = interactive(generate_ternary_plot, departamento=departamento_widget, nivel=nivel_widget)
interactive_plot


C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\1867058353.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ignac\AppData\Local\Temp\ipykernel_70008\1867058353.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



interactive(children=(Dropdown(description='Departamento', options=('Anta', 'Cachi', 'Cafayate', 'Capital', 'C…